In [17]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
import glob,os
from sklearn.preprocessing import  LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
IMAGE_SIZE = 256
BATCH_SIZE= 32
CHANNELS= 3
EPOCHS=50

In [19]:
dataset= tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle= True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size= BATCH_SIZE
)


Found 2152 files belonging to 3 classes.


In [20]:
class_names= dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [24]:
mild_dir=[]
moderate_dir=[]
no_dr_dir=[]
proliferate_dr_dir=[]
severe_dir=[]
for i in mild_path:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(128,128))
    img=img/255
    mild_dir.append(img)
for i in moderate_path:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(128,128))
    img=img/255
    moderate_dir.append(img)
for i in proliferate_dr_path:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(128,128))
    img=img/255
    proliferate_dr_dir.append(img)
for i in severe_path:
    img=cv2.imread(i,1)
    img=cv2.resize(img,(128,128))
    img=img/255
    severe_dir.append(img)

In [23]:
mild_path=glob.glob(r'gaussian_filtered_images/Mild/*.png')
moderate_path=glob.glob(r'gaussian_filtered_images/Moderate/*.png')
proliferate_dr_path=glob.glob(r'gaussian_filtered_images/Proliferate_DR/*.png')
severe_path=glob.glob(r'gaussian_filtered_images/Severe/*.png')

In [25]:
def data_import(mild_dir,moderate_dir,proliferate_dr_dir,severe_dir):
    df_data_mild = pd.DataFrame({'image':mild_dir, 'label': 'mild'})
    df_data_moderate = pd.DataFrame({'image':moderate_dir, 'label': 'moderate'})
    #df_data_no = pd.DataFrame({'image':no_dr_dir, 'label': 'no_DR'})
    df_data_proliferate = pd.DataFrame({'image':proliferate_dr_dir, 'label': 'proliferate_DR'})
    df_data_severe = pd.DataFrame({'image':severe_dir, 'label': 'severe'})

    
    final_data = [df_data_mild, df_data_moderate,df_data_proliferate,df_data_severe]
    final_data = pd.concat(final_data)
    
    all_data = final_data['image']
    labels = final_data['label']
    all_data=np.stack(all_data,axis=0)
    labels = LabelBinarizer().fit_transform(labels)
    return all_data,labels

In [26]:
dataset,labels   = data_import(mild_dir,moderate_dir,proliferate_dr_dir,severe_dir)

In [27]:
x=dataset
y = np.where(labels==1)[1]

from tensorflow.keras.utils import to_categorical
y_onehot = to_categorical(y)

x_train,x_test,y_train,y_test = train_test_split(x,y_onehot,test_size=0.25,stratify=y_onehot)

In [36]:
from keras.models import Sequential
from keras.layers import GaussianNoise
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout

base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(128,128,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy','AUC','Precision','Recall'])

In [38]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
44/44 [==============================] - 69s 1s/step - loss: 1.7983 - accuracy: 0.3082 - auc: 0.5560 - precision: 0.3152 - recall: 0.2162
Epoch 2/10
44/44 [==============================] - 47s 1s/step - loss: 1.5688 - accuracy: 0.3757 - auc: 0.6277 - precision: 0.4128 - recall: 0.2651
Epoch 3/10
44/44 [==============================] - 46s 1s/step - loss: 1.4244 - accuracy: 0.4425 - auc: 0.6789 - precision: 0.4995 - recall: 0.3427
Epoch 4/10
44/44 [==============================] - 46s 1s/step - loss: 1.4128 - accuracy: 0.4389 - auc: 0.6698 - precision: 0.4877 - recall: 0.3125
Epoch 5/10
44/44 [==============================] - 48s 1s/step - loss: 1.3342 - accuracy: 0.4727 - auc: 0.6892 - precision: 0.5180 - recall: 0.3211
Epoch 6/10
44/44 [==============================] - 47s 1s/step - loss: 1.3143 - accuracy: 0.4784 - auc: 0.6896 - precision: 0.5053 - recall: 0.3082
Epoch 7/10
44/44 [==============================] - 46s 1s/step - loss: 1.3304 - accuracy: 0.4864 - auc: 0

In [39]:
model.evaluate(x_train,y_train)

44/44 [==============================] - 51s 1s/step - loss: 1.4140 - accuracy: 0.2047 - auc: 0.4870 - precision: 0.0000e+00 - recall: 0.0000e+00


[1.4139556884765625, 0.20474137365818024, 0.4870026409626007, 0.0, 0.0]

In [40]:
model.evaluate(x_test,y_test)

15/15 [==============================] - 15s 1s/step - loss: 1.4116 - accuracy: 0.2086 - auc: 0.4908 - precision: 0.0000e+00 - recall: 0.0000e+00


[1.4116491079330444, 0.2086021453142166, 0.4907827377319336, 0.0, 0.0]